In [ ]:
# Un code python ressemblera à ça : 

import openturns as ot
from math import sqrt

# 1. The function G
def functionCrue(X) :
    Hd = 3.0
    Zb = 55.5
    L = 5.0e3
    B = 300.0
    Zd = Zb + Hd
    Q, Ks, Zv, Zm = X
    alpha = (Zm - Zv)/L
    H = (Q/(Ks*B*sqrt(alpha)))**(3.0/5.0)
    Zc = H + Zv
    S = Zc - Zd
    return [S]

# Creation of the problem function
g = ot.PythonFunction(4, 1, functionCrue) 
g = ot.MemoizeFunction(g)

# 2. Random vector definition
myParam = ot.GumbelAB(1013., 558.)
Q = ot.ParametrizedDistribution(myParam)
otLOW = ot.TruncatedDistribution.LOWER
Q = ot.TruncatedDistribution(Q, 0, otLOW)
Ks = ot.Normal(30.0, 7.5)
Ks = ot.TruncatedDistribution(Ks, 0, otLOW)
Zv = ot.Uniform(49.0, 51.0)
Zm = ot.Uniform(54.0, 56.0)

# 4. Create the joint distribution function 
distribution = ot.ComposedDistribution([Q, Ks, Zv, Zm])

mySobolSample = ot.SobolSample(distribution)
mySobolStudy = ot.IterativeSobolStudy(g)
# on peut imaginer qu'on peut passer en paramètre l'algo qu'on veut
# (avec Martinez par défaut)
# on peut aussi demander les ordres (1 par défaut)

# 3. Réalise les simulations
# les simulations sont réalisées à l'intérieur de la SobolIterativeStudy
# à partir de jeux de paramèters réalisés sur la ComposedDistribution
# et assemblés par le SobolSample
size = 50
# 3.1. Increment with several points 
# mySobolSample.getRealization() renvoie n+2 jeux de paramètres
for i in range(size):
    mySobolStudy.increment(mySobolSample.getRealization())

# 3.1. Increment with one sample 
# mySobolSample.getSample(size) renvoie size * (n+2) jeux de paramètres 
mySobolStudy.increment(mySobolSample.getSample(size))

# 4. Récupère tous les indices (tuple)
indices = mySobolStudy.getValues()
indice0 = indices[0]